# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [1]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [1]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import datetime
pd.set_option('display.max_rows', 1000)


def assignment2_1():
    data = 'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv'
    df = pd.read_csv(data) 
    # Replace string dates to actutal datetime format and sorting the dataframe by date
    df['Date'] = pd.to_datetime(df['Date'])
    df['New Date'] = df['Date'].dt.strftime('%m-%d')
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['Temp'] = df['Data_Value']*.1
    # Remove leap year days i.e. februrary 29th
    leap_year = df[(df['Month'] == 2) & (df['Day'] == 29)].index
    df = df.drop(leap_year)
    # Since the data only looks at year 2005-2014, we separate out year 2015
    df2015 = df[df['Year'] == 2015]
    # Reset the index
    df = df.drop(df2015.index)
    
    # Now, what we need to do is to construct a new data frame with just the highest temperature
    # of day the month among all the stations and all years
    df = df.groupby('New Date')['Temp'].agg({'max temp': np.max, 'min temp': np.min})
    df.rename(index = lambda x: pd.to_datetime(x, format = '%m-%d').replace(year=2015), inplace=True)
    df2015 = df2015.groupby('New Date')['Temp'].agg({'max temp': np.max, 'min temp': np.min})
    df2015.rename(index = lambda x: pd.to_datetime(x, format = '%m-%d').replace(year=2015), inplace=True)
    df_higher = df2015.loc[df2015['max temp'] > df['max temp'],:]
    df_lower = df2015.loc[df2015['min temp'] < df['min temp'],:]
    
    
    # Now lets try to plot the graph
    # Set the figure size
    fig = plt.figure(figsize = [20,10])
    
    # Plot the max and min temps for 2005-2014 and shade the area in between both lines
    plt.plot(df['min temp'], '-', color = '#1FBDC7' , label = 'Max Temp (2005-2014) by Day' )
    plt.plot(df['max temp'], '-', color = '#F77424', label = 'Min Temp (2005-2014) by Day')
    plt.fill_between(df.index, df['max temp'], df['min temp'], facecolor = '#D8D4D2', alpha=0.5,
                    label = 'Daily Temperature Range (2005-2014)')
    
    # Plot the extreme temps in 2015
    plt.plot(df_higher['max temp'], '^', color = '#E63313', label = 'Max Temp (2015 > 2005-2014)')
    plt.plot(df_lower['min temp'], 'v', color = '#0B3CE2', label = 'Min Temp (2015 > 2005-2014)')
    
    # Set the labels, title and legend of the graph
    plt.ylabel('Temperature ($^\circ$C)')
    plt.title('Extreme Temperatures in 2015 vs. Daily Max & Min Temperatures from 2005 to 2014\n'
              'at Ann Arbor, Michigan, United States')
    plt.legend()
    
    # Adjusting the x-axis and adding grid lines
#     ax = plt.gca()
#     monthformat = mdates.DateFormatter('%b')
#     months = mdates.MonthLocator()
#     ax.xaxis.set_major_locator(months)
#     ax.xaxis.set_major_formatter(monthformat)
#     ax.set_xlim([datetime.date(2015, 1, 1), datetime.date(2015, 12, 31)])
#     ax.grid(linestyle=':', alpha =0.5)
    
    # Save the figure
    fig.savefig('Assignment 2.png', dpi=300, bbox_inches='tight')
        
    
    return plt.show()


assignment2_1()

SpecificationError: nested renamer is not supported